# Import Packages

In [2]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader 

import torchvision.transforms as transforms 
import torchvision.datasets as dsets 

In [4]:
torch.manual_seed(0)

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                            train=False, 
                            transform=transforms.ToTensor(), 
                            download=True)

batch_size=100
n_iter = 3000
n_epochs = int(n_iter / (len(train_dataset) / batch_size))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Build Model

In [10]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

In [11]:
input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 
learning_rate = 1e-1

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)


In [20]:
iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

Iteration: 500, Loss: 0.1771, Accuracy: 93.87%
Iteration: 1000, Loss: 0.2288, Accuracy: 94.66%
Iteration: 1500, Loss: 0.1610, Accuracy: 95.23%
Iteration: 2000, Loss: 0.0962, Accuracy: 95.82%
Iteration: 2500, Loss: 0.1703, Accuracy: 96.17%
Iteration: 3000, Loss: 0.0795, Accuracy: 96.50%


- Models parameters: $\theta \in \mathcal{R}^d$
- Loss function: $J(\theta)$
- Gradient w.r.t. parameters: $\nabla J(\theta)$
- Learning rate: $\eta$
- Batch Gradient Descent: $\theta = \theta - \eta \cdot \nabla J(\theta)$
- Stochastic Gradient Descent: $\theta = \theta - \eta \cdot \nabla J(\theta, x^i, y^i)$
- Mini-Bacth Gradient Descent: $\theta = \theta - \eta \cdot \nabla J(\theta, x^{i:i+n}, y^{i:i+n})$

# SGD Momentum

- $v_t = \gamma v_{t-1} + \eta \cdot \nabla J(\theta, x^{i:i+n}, y^{i:i+n})$
- $\theta = \theta - v_t$

In [23]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 
learning_rate = 1e-1

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.1375, Accuracy: 95.22%
Iteration: 1000, Loss: 0.0629, Accuracy: 96.55%
Iteration: 1500, Loss: 0.0233, Accuracy: 97.36%
Iteration: 2000, Loss: 0.1141, Accuracy: 97.00%
Iteration: 2500, Loss: 0.0622, Accuracy: 97.42%
Iteration: 3000, Loss: 0.1135, Accuracy: 97.39%


# SGD nesterov momentum

- $v_t = \gamma v_{t-1} + \eta \cdot \nabla J(\theta - \gamma v_{t-1}, x^{i:i+n}, y^{i:i+n})$ 
- $\theta = \theta - v_t $


In [26]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 
learning_rate = 1e-1

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.1189, Accuracy: 96.16%
Iteration: 1000, Loss: 0.1993, Accuracy: 96.66%
Iteration: 1500, Loss: 0.1848, Accuracy: 96.92%
Iteration: 2000, Loss: 0.0597, Accuracy: 97.19%
Iteration: 2500, Loss: 0.0826, Accuracy: 97.71%
Iteration: 3000, Loss: 0.1586, Accuracy: 97.39%


# Adam 

- $m_t = \beta_1 m_{t-1} + (1-\beta_1)g_t$
- $v_t = \beta_2 v_{t-1} + (1-\beta_2)g^2_t$
- $\hat{m}_t = \frac{m_t}{1-\beta_2}$
- $\hat{v}_t = \frac{v_t}{1-\beta_2}$
- $\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v}_t}+\epsilon}\cdot \hat{m}_t$

In [27]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters()) # learning_rate=0.01

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.3647, Accuracy: 94.01%
Iteration: 1000, Loss: 0.1764, Accuracy: 95.37%
Iteration: 1500, Loss: 0.0639, Accuracy: 96.45%
Iteration: 2000, Loss: 0.0908, Accuracy: 96.85%
Iteration: 2500, Loss: 0.1346, Accuracy: 96.97%
Iteration: 3000, Loss: 0.1018, Accuracy: 97.14%


# Adagrad

In [28]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters()) # learning_rate = 0.01

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.1597, Accuracy: 93.70%
Iteration: 1000, Loss: 0.1298, Accuracy: 94.70%
Iteration: 1500, Loss: 0.1998, Accuracy: 95.34%
Iteration: 2000, Loss: 0.1116, Accuracy: 95.66%
Iteration: 2500, Loss: 0.1493, Accuracy: 95.86%
Iteration: 3000, Loss: 0.1239, Accuracy: 96.20%


# Adadelta

In [29]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters()) # learning_rate = 0.01

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.0812, Accuracy: 94.97%
Iteration: 1000, Loss: 0.1112, Accuracy: 96.39%
Iteration: 1500, Loss: 0.0638, Accuracy: 97.08%
Iteration: 2000, Loss: 0.0848, Accuracy: 97.07%
Iteration: 2500, Loss: 0.1084, Accuracy: 97.28%
Iteration: 3000, Loss: 0.1699, Accuracy: 96.80%


# Adamax

In [30]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 
learning_rate = 1e-1

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters()) # learning_rate = 0.01

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.1205, Accuracy: 94.64%
Iteration: 1000, Loss: 0.0881, Accuracy: 95.98%
Iteration: 1500, Loss: 0.1541, Accuracy: 96.85%
Iteration: 2000, Loss: 0.0592, Accuracy: 97.47%
Iteration: 2500, Loss: 0.0624, Accuracy: 97.58%
Iteration: 3000, Loss: 0.0362, Accuracy: 97.50%


# RMSProp

In [31]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, output_dim) 
        self.relu = nn.ReLU() 
        
        self._init_weight()
    
    def forward(self, images):
        images = images.view(-1, self.input_dim)
        
        out = self.fc1(images) 
        out = self.relu(out) 
        
        out = self.fc2(out) 
        return out 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(f'initializing... {module}')
                nn.init.kaiming_normal_(module.weight) 
                nn.init.zeros_(module.bias)               

input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 

model = FFNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters()) # learning_rate = 0.01

iter = 0 
for epoch in range(1, n_epochs+1):
    
    model.train()
    for (images, labels) in train_loader:
        
        optimizer.zero_grad()
        pred_y = model(images)
        loss = criterion(pred_y, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct, total = 0, 0
            
            model.eval()
            for (images, labels) in test_loader:
                
                pred_y = model(images)
                loss = criterion(pred_y, labels)
                pred_y = torch.argmax(pred_y, dim=1)
                
                total += len(labels)
                correct += (pred_y == labels).sum()
            
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

initializing... Linear(in_features=784, out_features=100, bias=True)
initializing... Linear(in_features=100, out_features=10, bias=True)
Iteration: 500, Loss: 0.1782, Accuracy: 95.18%
Iteration: 1000, Loss: 0.2938, Accuracy: 96.60%
Iteration: 1500, Loss: 0.0240, Accuracy: 96.46%
Iteration: 2000, Loss: 0.0837, Accuracy: 95.75%
Iteration: 2500, Loss: 0.0755, Accuracy: 96.71%
Iteration: 3000, Loss: 0.2874, Accuracy: 96.60%
